Items to handle:
* No seperate validation data given (**Solution:** using subset for trainingdata generation)
* Elements in each catagores are different

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
import seaborn as sns
%matplotlib inline
import os
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from   tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
character = mpimg.imread('../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/nelson_muntz_25.jpg')
plt.imshow(character)
plt.show()

In [ ]:
train_dir = '../input/the-simpsons-characters-dataset/simpsons_dataset/simpsons_dataset/'
dir_df = pd.DataFrame({'class': [x for x in os.listdir(train_dir)]})
character_class = np.array(dir_df.sort_values(by=['class'], ascending = True))
character_class = character_class.reshape(42)
character_class

In [ ]:
test_path = '../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/'
test_images = [x for x in os.walk(test_path)]
test_fnames = os.listdir( test_path )
test_fnames[:4]

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',input_shape=(250,250,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(42,activation='softmax'),
])
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

Images Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(validation_split=0.3,
                            rescale = 1.0/255.)

In [ ]:
train_generator = datagen.flow_from_directory(
    directory=train_dir,
    target_size=(250, 250),
    color_mode="rgb",
    batch_size=20,
    class_mode="categorical",
    subset='training', 
)

validation_generator = datagen.flow_from_directory(
    directory=train_dir,
    target_size=(250, 250),
    color_mode="rgb",
    batch_size=20,
    class_mode="categorical",
    subset='validation',
)

In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator, 
                    steps_per_epoch=100,
                    validation_steps=50,
                    epochs=20,
                    verbose=2)

In [ ]:
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
test_image_files = [os.path.join(test_path, f) for f in test_fnames]
test_image_path = random.choice(test_image_files)
test_image = load_img(test_image_path, target_size=(250, 250))
print('Original toon name:','_'.join(test_image_path.split('/')[-1].split('.')[0].split('_')[:-1]))
x   = img_to_array(test_image) 
x   = x.reshape((1,) + x.shape)
x /= 255.0
y_prob = model.predict(x) 
y_classes = y_prob.argmax()
print("Predicted toon name:",character_class[y_classes])
plt.imshow(test_image)
plt.show()

In [ ]:
total_mached = 0 
failed_labels = []
for test_image in test_image_files:
    org_label = '_'.join(test_image.split('/')[-1].split('_')[:-1])
    test_img_load = load_img(test_image, target_size=(250,250))
    
    x = img_to_array(test_img_load)
    x = x.reshape((1,) + x.shape)
    x /= 255.0
    y_prob = model.predict(x)
    y_cls = y_prob.argmax()
    pred_label = character_class[y_cls]
    
    if org_label == pred_label:
        total_mached +=1
    else:
        failed_labels.append([org_label,pred_label,test_image])
    
print('Total mached:', total_mached)
print('Test Accuracy:', total_mached/len(test_image_files))

In [ ]:
len(failed_labels)

In [ ]:
for i in range(len(failed_labels)):
    character = mpimg.imread(falid_labels[i][2])
    print(f'Original:{failed_labels[i][0]}, Predict: {failed_labels[i][1]}')
    plt.imshow(character)
    plt.show()

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

In [ ]:
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

img_path = random.choice(test_image_files)
img = load_img(img_path, target_size=(250, 250))  # this is a PIL image

x   = img_to_array(img)                           
x   = x.reshape((1,) + x.shape)                   
x /= 255.0

successive_feature_maps = visualization_model.predict(x)
layer_names = [layer.name for layer in model.layers]

# -----------------------------------------------------------------------
# Now let's display our representations
# -----------------------------------------------------------------------
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    #-------------------------------------------
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------

    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' ) 
    
plt.show()

Adding Augmentation and Dropout

In [ ]:
model_2 = keras.Sequential([
    keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(250,250,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,(3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128,(3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128,(3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(42, activation='softmax')
])

In [ ]:
model_2.summary()

In [ ]:
model_2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

In [ ]:
datagen_2 = ImageDataGenerator(validation_split=0.3,
                             rescale = 1.0/255.,
                             rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

In [ ]:
train_generator_2 = datagen_2.flow_from_directory(
    directory=train_dir,
    target_size=(250, 250),
    color_mode="rgb",
    batch_size=20,
    class_mode="categorical",
    subset='training', 
)

validation_generator_2 = datagen_2.flow_from_directory(
    directory=train_dir,
    target_size=(250, 250),
    color_mode="rgb",
    batch_size=20,
    class_mode="categorical",
    subset='validation',
)

In [ ]:
history_2 = model_2.fit(train_generator_2,
                    validation_data=validation_generator_2, 
                    steps_per_epoch=100,
                    validation_steps=50,
                    epochs=100,
                    verbose=2)

In [ ]:
acc = history_2.history['accuracy']
val_acc = history_2.history['val_accuracy']
loss = history_2.history['loss']
val_loss = history_2.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_image_files = [os.path.join(test_path, f) for f in test_fnames]
test_image_path = random.choice(test_image_files)
test_image = load_img(test_image_path, target_size=(250, 250))
print('Original toon name:','_'.join(test_image_path.split('/')[-1].split('.')[0].split('_')[:-1]))
x   = img_to_array(test_image) 
x   = x.reshape((1,) + x.shape)
x /= 255.0
y_prob = model_2.predict(x) 
y_classes = y_prob.argmax()
print("Predicted toon name:",character_class[y_classes])
plt.imshow(test_image)
plt.show()

In [ ]:
total_mached = 0 
failed_labels = []
for test_image in test_image_files:
    org_label = '_'.join(test_image.split('/')[-1].split('_')[:-1])
    test_img_load = load_img(test_image, target_size=(250,250))
    
    x = img_to_array(test_img_load)
    x = x.reshape((1,) + x.shape)
    x /= 255.0
    y_prob = model_2.predict(x)
    y_cls = y_prob.argmax()
    pred_label = character_class[y_cls]
    
    if org_label == pred_label:
        total_mached +=1
    else:
        failed_labels.append([org_label,pred_label,test_image])
    
print('Total mached:', total_mached)
print('Test Accuracy:', total_mached/len(test_image_files))

In [ ]:
len(failed_labels)

In [ ]:
for i in range(len(failed_labels)):
    character = mpimg.imread(failed_labels[i][2])
    print(f'Original:{failed_labels[i][0]}, Predict: {failed_labels[i][1]}')
    plt.imshow(character)
    plt.show()

In [ ]:
history_2 = model_2.fit(train_generator_2,
                    validation_data=validation_generator, 
                    steps_per_epoch=100,
                    validation_steps=50,
                    epochs=100,
                    verbose=2)

In [ ]:
acc = history_2.history['accuracy']
val_acc = history_2.history['val_accuracy']
loss = history_2.history['loss']
val_loss = history_2.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_image_files = [os.path.join(test_path, f) for f in test_fnames]
test_image_path = random.choice(test_image_files)
test_image = load_img(test_image_path, target_size=(250, 250))
print('Original toon name:','_'.join(test_image_path.split('/')[-1].split('.')[0].split('_')[:-1]))
x   = img_to_array(test_image) 
x   = x.reshape((1,) + x.shape)
x /= 255.0
y_prob = model_2.predict(x) 
y_classes = y_prob.argmax()
print("Predicted toon name:",character_class[y_classes])
plt.imshow(test_image)
plt.show()

In [ ]:
total_mached = 0 
failed_labels = []
for test_image in test_image_files:
    org_label = '_'.join(test_image.split('/')[-1].split('_')[:-1])
    test_img_load = load_img(test_image, target_size=(250,250))
    
    x = img_to_array(test_img_load)
    x = x.reshape((1,) + x.shape)
    x /= 255.0
    y_prob = model_2.predict(x)
    y_cls = y_prob.argmax()
    pred_label = character_class[y_cls]
    
    if org_label == pred_label:
        total_mached +=1
    else:
        failed_labels.append([org_label,pred_label,test_image])
    
print('Total mached:', total_mached)
print('Test Accuracy:', total_mached/len(test_image_files))
print('Total miss classified:', len(failed_labels))

In [ ]:
for i in range(len(failed_labels)):
    character = mpimg.imread(failed_labels[i][2])
    print(f'Original:{failed_labels[i][0]}, Predict: {failed_labels[i][1]}')
    plt.imshow(character)
    plt.show()

In [ ]:
datagen_3 = ImageDataGenerator(validation_split=0.3,
                             rescale = 1.0/255.,
                             rotation_range=15,
                             width_shift_range=0.15,
                             height_shift_range=0.15,
                             shear_range=0.15,
                             zoom_range=0.15,
                             channel_shift_range=1,
                             horizontal_flip=True,
                             vertical_flip=True,
                             fill_mode='nearest')